# Face Detection Multi-Model OpenVINO Model Server Deployment in OpenShift

We will show you how to deploy OpenVINO Model Server (OVMS) service with multiple models in an OpenShift cluster. We will run a face detection request to the AI inference service which will return age, gender, and emotion recognition as an ouput for each detected face.

Requirements:
- OpenShift cluster with the API access to a project
- installed [OpenVINO Model Server Operator](https://catalog.redhat.com/software/operators/search?q=openvino)
- JupyterLab environment with Python3 deployed in the cluster

If you don't have an OpenShift account, you can sign up for 30 or 60 day [free trial of Red Hat OpenShift](https://www.openshift.com/try).

## Login to OpenShift with API Token

First, let's login to OpenShift cluster using `oc` tool. 

In the Red Hat OpenShift console, click on your username and select `Copy login command`.

![copy-login.png](notebook-files/copy-login.png)

Click on `Display Token` and your API token will appear.

![log-in-with-token.png](notebook-files/log-in-with-token.png)

Copy `Log in with token` command and paste it in the cell below. The command has your `<user-API-token>` and `<cluster-DNS-name>`.

In [ ]:
!oc login --token=<user-API-token> --server=https://api.<cluster-DNS-name>:6443

Create `ovms` project and go to this project.

In [ ]:
!oc new-project ovms
!oc project ovms

## Create MinIO Storage

OpenVINO Model Server exposes DL models over gRPC and REST interface. The models can be stored in cloud storage like AWS S3, Google Storage or Azure Blobs. In OpenShift and Kubernetes, Persistent Storage Claim could be used as well. In this tutorial, we will use MinIO service which is an equivalent of AWS S3.

Let's create a MinIO service.

Now deploy Minio service. Note that the configuration below creates Minio server with emphemeral storage which will be deleted each time the pod is restarted. It includes also the default credentials. All in all, it is only a demonstrative purpose.

In [ ]:
!oc apply -f minio.yaml

Next step is to download `mc`, MinIO Client.

In [ ]:
!wget https://dl.min.io/client/mc/release/linux-amd64/mc

Change the access permissions on `mc`, so we can run commands with it.

In [ ]:
!chmod 755 mc

Let's make an alias for the MinIO service.

In [ ]:
!./mc alias set minio http://minio-service.ovms:9000 minio minio123

Create a `minio/models` bucket; it's where we will store our models.

In [ ]:
!./mc mb minio/models

## Create Model Repository

Now, we will upload the models to the MinIO bucket for serving in the OpenVINO Model Server. We will use 3 models:
* [face detection](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/intel/face-detection-retail-0004/description/face-detection-retail-0004.md)
* [age gender recognition](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/intel/age-gender-recognition-retail-0013/description/age-gender-recognition-retail-0013.md)
* [emotion recognition](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/intel/emotions-recognition-retail-0003/description/emotions-recognition-retail-0003.md)

First, we will download the models here.

In [ ]:
!curl --create-dirs https://storage.openvinotoolkit.org/repositories/open_model_zoo/2021.3/models_bin/2/age-gender-recognition-retail-0013/FP32/age-gender-recognition-retail-0013.xml -o age-gender/1/age-gender-recognition-retail-0013.xml 
!curl --create-dirs https://storage.openvinotoolkit.org/repositories/open_model_zoo/2021.3/models_bin/2/age-gender-recognition-retail-0013/FP32/age-gender-recognition-retail-0013.bin -o age-gender/1/age-gender-recognition-retail-0013.bin
!curl --create-dirs https://storage.openvinotoolkit.org/repositories/open_model_zoo/2021.3/models_bin/2/face-detection-retail-0004/FP32/face-detection-retail-0004.xml -o face-detection/1/face-detection-retail-0004.xml
!curl --create-dirs https://storage.openvinotoolkit.org/repositories/open_model_zoo/2021.3/models_bin/2/face-detection-retail-0004/FP32/face-detection-retail-0004.bin -o face-detection/1/face-detection-retail-0004.bin
!curl --create-dirs https://storage.openvinotoolkit.org/repositories/open_model_zoo/2021.3/models_bin/2/emotions-recognition-retail-0003/FP32/emotions-recognition-retail-0003.xml -o emotions/1/emotions-recognition-retail-0003.xml
!curl --create-dirs https://storage.openvinotoolkit.org/repositories/open_model_zoo/2021.3/models_bin/2/emotions-recognition-retail-0003/FP32/emotions-recognition-retail-0003.bin -o emotions/1/emotions-recognition-retail-0003.bin

Now, copy the models into the MinIO bucket.

In [ ]:
!./mc cp --recursive age-gender minio/models/
!./mc cp --recursive face-detection minio/models/
!./mc cp --recursive emotions minio/models/

Let's make sure the models have been successfully copied.

In [ ]:
!./mc ls -r minio/models

## Create a Directed Acyclic Graph Pipeline

To implement the multi-model pipeline, we will use a Directed Acyclic Graph, or DAG, scheduler. Here's the workflow.

![graph](notebook-files/faces_analysis_graph.svg)

As you can see from the workflow, DAG is a graph that doesn't have any loops. It consists of processes that are only moving forward.

We will need the `config.json` to define the DAG pipeline. 

In [ ]:
!cat config.json

## Create a Custom Node

We will use [this custom node](https://github.com/openvinotoolkit/model_server/tree/develop/src/custom_nodes/model_zoo_intel_object_detection) which will analyze the response from the face detection model. Based on the inference results and the input image, the custom node will generate a list of detected boxes. Each image in the output will be resized to the predefined target size to fit the input of the next model in the DAG pipeline. In addition to detected boxes, the results include the coordinates and the detection scores. 

The main functionality of custom node in the OVMS DAG scheduler is that it allows us to create an arbitrary implementation of the data transformation node in the pipeline. It will be attached to the Model Server as a dynamic library. 

Clone the Model Server repo and download OpenCV archive.

In [ ]:
!git clone --depth=1 -b develop https://github.com/openvinotoolkit/model_server
!curl -s https://download.01.org/opencv/master/openvinotoolkit/thirdparty/linux/opencv/opencv_4.5.1-044_centos7.txz | tar --use-compress-program=xz -xf -

Run commands below to compile `libcustom_node.so`, the custom node library. 

In [ ]:
!g++ -c -std=c++17 model_server/src/custom_nodes/model_zoo_intel_object_detection/model_zoo_intel_object_detection.cpp -fpic  -I./opencv/include/ -Wall -Wno-unknown-pragmas -Werror -fno-strict-overflow -fno-delete-null-pointer-checks -fwrapv -fstack-protector
!g++ -shared -o libcustom_node.so model_zoo_intel_object_detection.o -L./opencv/lib/ -I./opencv/include/ -lopencv_core -lopencv_imgproc -lopencv_imgcodecs

Let's check if the custom node library was created.

In [ ]:
!ls -l libcustom_node.so

## Deploy OpenVINO Model Server with a Multi-Model Pipeline

Let's add the custom node library and `config.json` to a ConfigMap resource. Later, it will be mounted inside the OVMS service.

In [ ]:
!oc create configmap ovms-face-detection-pipeline \
                    --from-file=libcustom_node.so=libcustom_node.so \
                    --from-file=config.json=config.json

Here's the yaml file used to configure the OVMS service. We specified its name to be `ovms-pipeline`.

In [ ]:
!cat ovms-face-detection-pipeline.yaml

Let's create the `ovms-pipeline` service.

In [ ]:
!oc apply -f ovms-face-detection-pipeline.yaml

Let's see if pod and service were created. They should start with `ovms-pipeline`.

In [ ]:
!oc get pod
!oc get service

Let's check if the OpenVINO Model Server service is running by making an API request via cURL. Models' `state` should be `AVAILABLE`.

In [ ]:
!curl -s http://ovms-pipeline.ovms.svc:8081/v1/config

## Run an Inference Request

The pipeline execution is represented as the `find_face_images` model. The client runs an inference request exactly the same way as it would run with a single model.

Let's import Python packages needed for inference request.

In [ ]:
import grpc
import cv2
import os
import numpy as np
from tensorflow import make_tensor_proto, make_ndarray
import argparse
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc
from IPython.display import Image, display

We will run face detection on this image.

![people](people.jpg)


Prepare the image for sending it in the gRPC request to the `ovms-pipeline` service.

In [ ]:
img = cv2.imread('people.jpg').astype(np.float32)  # BGR color format, shape HWC
resolution = (400, 600)
img = cv2.resize(img, (resolution[1], resolution[0]))
img = img.transpose(2,0,1).reshape(1,3,resolution[0],resolution[1])

Next, let's establish connection with the `ovms-pipeline` service.

In [ ]:
address = "ovms-pipeline.ovms.svc:8080"
MAX_MESSAGE_LENGTH = 1024 * 1024 * 8  # incresed default max size of the message
channel = grpc.insecure_channel(address,
    options=[
        ('grpc.max_send_message_length', MAX_MESSAGE_LENGTH),
        ('grpc.max_receive_message_length', MAX_MESSAGE_LENGTH),
    ])

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
request = predict_pb2.PredictRequest()
request.model_spec.name = "find_face_images"

Send the request and prediction will be executed. Note that the exception is handled when the pipeline doesn't detect any face in the image. 

In [ ]:
request.inputs['image'].CopyFrom(make_tensor_proto(img, shape=img.shape))
try:
    response = stub.Predict(request, 10.0)
except grpc.RpcError as err:
    if err.code() == grpc.StatusCode.ABORTED:
        print('No face has been found in the image')
        exit(1)
    else:
        raise err


We will receive results as `ages`, `genders`, `emotions`, and `face_coordinates` in `response` object. `face_images` output returns cropped faces retrieved from the original image.

Define functions to process the output.

In [ ]:
images = []
def save_face_images_as_jpgs(output_nd, name, location):
    for i in range(output_nd.shape[0]):
        out = output_nd[i][0]
        out = out.transpose(1,2,0)
        output_file_name = name + '_' + str(i) + '.jpg'
        cv2.imwrite(os.path.join(location, output_file_name), out)
        images.append(output_file_name)
        
def update_people_ages(output_nd, people):
    for i in range(output_nd.shape[0]):
        age = int(output_nd[i,0,0,0,0] * 100)
        if len(people) < i + 1:
            people.append({'age': age})
        else:
            people[i].update({'age': age})
    return people

def update_people_genders(output_nd, people):
    for i in range(output_nd.shape[0]):
        gender = 'male' if output_nd[i,0,0,0,0] < output_nd[i,0,1,0,0] else 'female'
        if len(people) < i + 1:
            people.append({'gender': gender})
        else:
            people[i].update({'gender': gender})
    return people

def update_people_emotions(output_nd, people):
    emotion_names = {
        0: 'neutral',
        1: 'happy',
        2: 'sad',
        3: 'surprised',
        4: 'angry'
    }
    for i in range(output_nd.shape[0]):
        emotion_id = np.argmax(output_nd[i,0,:,0,0])
        emotion = emotion_names[emotion_id]
        if len(people) < i + 1:
            people.append({'emotion': emotion})
        else:
            people[i].update({'emotion': emotion})
    return people

def update_people_coordinate(output_nd, people):
    for i in range(output_nd.shape[0]):
        if len(people) < i + 1:
            people.append({'coordinate': output_nd[i,0,:]})
        else:
            people[i].update({'coordinate': output_nd[i,0,:]})
    return people

Let's process the output.

In [ ]:
people = []

for name in response.outputs:
    print(f"Output: name[{name}]")
    tensor_proto = response.outputs[name]
    output_nd = make_ndarray(tensor_proto)
    print(f"    numpy => shape[{output_nd.shape}] data[{output_nd.dtype}]")

    if name == 'face_images':
        save_face_images_as_jpgs(output_nd, name, ".")
    if name == 'ages':
        people = update_people_ages(output_nd, people)
    if name == 'genders':
        people = update_people_genders(output_nd, people)
    if name == 'emotions':
        people = update_people_emotions(output_nd, people)
    if name == 'face_coordinates':
        people = update_people_coordinate(output_nd, people)

Now we can view the results.

In [ ]:
print('\nFound', len(people), 'faces:\n')
for num, person in enumerate(people):
    display(Image(images[num]))
    print(f"""
Estimated Age: {person['age']}
Estimated Gender: {person['gender']}
Estimated Emotion: {person['emotion']}
Original Image Coordinates:{person['coordinate']}\n""")

## Cleanup

Let's free up resources.

In [ ]:
!oc delete ovms ovms-pipeline
!oc delete deploy minio
!oc delete service minio-service
!oc delete configmap ovms-face-detection-pipeline

In [ ]:
!rm -rf age-gender
!rm -rf emotions
!rm -rf face-detection
!rm mc

## Next Steps

In this notebook, you have learned how to deploy an OVMS service with multiple models in an OpenShift cluster. Next, you can explore other OVMS notebooks:

- [Deploy Image Classification with OpenVINO Model Server in OpenShift](../401-model-serving-openshift-resnet/ovms-openshift-resnet.ipynb)
- [Send gRPC and API Calls via Python Scripts to OpenVINO Model Server in OpenShift](../402-model-serving-openshift-python-scripts/ovms-openshift-python-scripts.ipynb)
